# Kundenabwanderung

Kundenabwanderung ist ein branchenübergreifendes Problem.
Häufig wird sehr viel Energie in die Gewinnung von Kunden gesteckt, aber das Halten der Kunden wird vernachlässigt.
Mit einer Prognose zur Kundenabwanderung können diejenigen Kunden identifiziert werden, welche gegebenenfalls abwandern würden.
Diese kann das Unternehmen dann mit besonderen Angeboten oder anderweitigen Aufmerksamkeiten vielleicht zurückgewinnen.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Nun laden wir die Datei `CustomerChurn.xlsx` herunter.
Der Code dafür ist bereits vorgegeben.
IBM stellt den Datensatz
[in einem Blogbeitrag](https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113)
vor.

In [ ]:
import os
import requests
import shutil

file_name = "CustomerChurn.xlsx"
if file_name not in os.listdir("."):
    print(f"Lade Datei '{file_name}' herunter")
    r = requests.get("https://public.dhe.ibm.com/software/data/sw-library/cognos/mobile/C11/data/CustomerChurn.xlsx", stream=True)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
else:
    print(f"Datei '{file_name}' ist bereits heruntergeladen")

## Einlesen der Daten

Das Modul `pandas` liefert die Möglichkeit, Excel-Dateien mit der Funktion `read_excel()` einzulesen. 
Weitere Informationen dazu finden Sie unter dem folgenden Link:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

<span style="color:blue">(a) Laden Sie Datei `CustomerChurn.xlsx` als ein DataFrame.
    Nennen Sie die Variable für das DataFrame `df`.
    Lassen Sie sich den DataFrame ausgeben.
</span> 

In [ ]:
# Ihr Code...
df = ...

Es wurden verschiedene Skalen für Attribute vorgestellt.
Die Methode `df.info()` zeigt an, welcher Typ innerhalb von pandas ermittelt worden ist.
Eine kurze technische Erklärung der Typen gibt es
[im Handbuch](https://pandas.pydata.org/docs/user_guide/basics.html#dtypes).

<span style="color:blue">(b) Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
    Welches Attribut wird gerade auf welcher Skala abgetragen?
    Welche Skala sollte das jeweilige Attribut eigentlich haben?
</span>

In [ ]:
df.info()

## Umwandlung in korrekte Typen

Nun möchten wir aus dem Text in der Spalte `Total Charges` Zahlenwerte gewinnen, die wir weiter verwenden können.

In [ ]:
df["Total Charges"] = pd.to_numeric(df['Total Charges'], errors='coerce')
df

<span style="color:blue">(c) Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
    Was hat sich verändert?
</span>

In [ ]:
df.info()

Spoiler Alert: Später im Jupyter Notebook möchten wir noch Maschinelles Lernen verwenden.
Deswegen wollen wir schon mal alle "Yes"-Einträge in Einsen und alle "No"-Einträge in Nullen umwandeln.
Damit Sie beruhigt den Code auch mehrfach aufrufen können, werden bereits erfolgte Konversionen beibehalten (1 bleibt 1 und 0 bleibt 0).
Hier ist das Beispiel-Codeschnippsel:

```python
df['MY_ATTRIBUTE'] = df['MY_ATTRIBUTE'].map({'Yes': 1, 'No': 0, 1: 1, 0: 0})
```

<span style="color:blue">(d) Wandeln sie für alle Spalte alle "Yes"- und "No"-Einträge um.
    Speichern Sie die neue Version wieder im DataFrame `df`.
</span>

In [ ]:
# Ihr Code...

Das Umwandeln der Vertragsarten und der Bezahlmethode sind komplexer.
Deswegen überspringen wir das für diese Übung und löschen die Werte einfach.

In [ ]:
df.drop(columns=["Payment Method", "Contract", "Internet Service"], inplace=True)
df

## Analyse des Datensatzes

Nach dem Speichern der Daten in einem `DataFrame` sowie der Vorverarbeitung dieser, können diese analysiert werden. 

## Grundlegende Wahrscheinlichkeit, dass ein Kunde abspringt

Das Modul `pandas` stellt eine Reihe von deskriptiven Analysewerkzeugen zur Verfügung, bspw. die Berechnung von Mittelwerten, Varianzen und Standardabweichungen einzelner Spalten.
Weiterführende Informationen dazu sind in der folgenden Dokumentation zu finden:
https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats.

Zum Beispiel gibt die nachfolgende Anweisung die Wahrscheinlichkeit, dass ein Kunde abspringt, aus.

<span style="color:blue">(e) Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
</span>

In [ ]:
df['Churn'].describe()

<span style="color:blue"> (f) Rufen Sie die Methode `.describe()` des DataFrames auf und interpretieren Sie die Ergebnisse.
</span>

In [ ]:
# Ihr Code...

## Eine ML-basierte Prognose

Mit der ersten Analyse konnten aus den Daten einige ziemlich einfache Schlüsse gezogen werden. 
In der Aufgabe 4 soll ein Algorithmus entwickelt werden, der vorhersagt, ob ein bestimmter Kunde abwandern würde.
Hier kommt das maschinelle Lernen ins Spiel:
Es wird ein Programm entwickelt, das aus den Beispieldaten lernt, ob ein bestimmter Kunde abwandern würde.

Für das maschinelle Lernen werden die Daten in einem ersten Schritt in zwei Arrays geteilt:
`X` enthält alle Werte, aus denen der Zusammenhang gelernt werden soll,
und `y` enthält nur den Wert der Spalte `Churn` für diese Zeile.
Die Klassifikationsalgorithmen vergleichen die Attributwerte von `X` mit den entsprechenden Werten von `y`,
um Muster zu erkennen, wie unterschiedliche Attributwerte das Abwandern eines Kunden vorhersagen

In [ ]:
X = df.drop(
    ['Customer ID', 'LoyaltyID', 'Churn'],  # Nenne alle Spalten, die nicht zum Lernen verwendet werden dürfen/sollen
    axis=1
).values
y = df['Churn'].values  # Das zu erlernene Ziel

In [ ]:
from sklearn import tree
clf_dt = tree.DecisionTreeClassifier(max_depth=3)

In einem nächsten Schritt wird der Entscheidungsbaum anhand der Trainingsdaten mit Hilfe der ```fit()```-Methode trainiert (http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.fit). 

In [ ]:
clf_dt.fit(X, y)

Gerade eben kam es vermutlich zu einer Fehlermeldung (genauer gesagt, einem `ValueError`).
Der Entscheidungsbaum von scikit-learn braucht nicht nur überall Zahlenwerte, sondern es dürfen auch keine Daten fehlen.
Deswegen verwerfen wir kurzerhand alle Zeilen, in denen wir fehlende Werte haben.

In [ ]:
df.dropna(axis=1, how='any', inplace=True)
df

Nun rufen wir das Lernverfahren erneut auf:

In [ ]:
X = df.drop(['Customer ID', 'LoyaltyID', 'Churn'], axis=1).values
y = df['Churn'].values

clf_dt.fit(X, y)

Diesmal sollte es geklappt haben.
Falls nicht, haben Sie vermutlich bei (d) etwas übersehen.

Jetzt, da der Entscheidungsbaum fertig ist, kann er mit Hilfe der Testdaten auf seine Güte hin bewertet werden.
Eine Möglichkeit zur Überprüfung des Modells stellt die `.score()`-Methode dar,
welche bei Klassifikationsproblemen die mittlere Genauigkeit (_Accuracy_) der angegebenen Testdaten ermittelt.

Weitere Informationen zu der `score()`-Methode sind unter folgendem Link zu finden:
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.score

In [ ]:
clf_dt.score(X, y)

<span style="color:blue">(g) Setzen Sie das Ergebnis in den Kontext, was wir bereits allein aufgrund des arithmetischen Mittels wissen.
    Würden Sie dieses Modell in der Praxis einsetzen?
</span>

In [ ]:
print("Wahrscheinlichkeit, dass der Kunde abwandert", df['Churn'].mean() * 100, "%")
print("Wahrscheinlichkeit, dass der Kunde nicht abwandert", (1 - df['Churn'].mean()) * 100, "%")

Der folgende Code visualisierung den Baum

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
tree.plot_tree(
    clf_dt,
    ax=ax,
    feature_names=df.drop(['Customer ID', 'LoyaltyID', 'Churn'], axis=1).columns
)
plt.show()

Die Boxen in der untersten Zeile stellen die Blätter da, die anderen Boxen sind die Knoten des Baumes.
In diesen Boxen ist die jeweils oberste Zeile die Bedingung, die überprüft wird.
Ist die Bedingung wahr, so wird der linke Pfad gewählt und ist die Bedigung falsch, so wird der rechte Pfad gewählt.

- Die Zeile mit dem `gini`-Wert gibt die Verteilung von abgewanderten zu beim Kunden verbliebenen Kunden beim jeweiligen Knoten bzw. Blatt an.
- Die Zeile mit dem `sample`-Wert gibt an, wie viele Zeilen der Daten (ohne Testdaten) den jeweiligen Knoten bzw. das jeweilige Blatt erreicht haben.
- Die Zeile mit dem `value`-Wert gibt an, wie viele Passagiere (ohne Testdaten) des jeweiligen Knotens bzw. Blattes als Kunden geblieben sind (1. Wert) bzw. abgewandert sind (2. Wert).
- Bei den Blättern gibt der größere `value`-Wert an, welche Prognose der Algorithmus für neue Daten trifft.

<span style="color:blue">(h) Ist diese Art von Entscheidungsbaum zu erwarten gewesen?
    Gibt es Unterscheidungen und Prognosen, die überraschend sind? Woran liegt dies?
</span>